In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import math
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from datetime import date, datetime
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
plt.style.use("seaborn")
sns.set(style="white")

## Perfil de peliculas y usuarios.

### Perfil de las peliculas segun su genero 

In [2]:
df_genres = pd.read_csv("./data/movie_genres.csv")
df_movies = pd.read_csv("./data/movies.csv")
df_ratings = pd.read_csv("./data/ratings_train.csv")
df_ratings_test = pd.read_csv("./data/ratings_test.csv")

In [3]:
# Creo una matriz donde se agrego un flag "1" en cada genero del que la pelicula pertenece.
# Perfil de la pelicula sobre el Genero
for g in df_genres["genre"].unique():
    mids = df_genres.loc[df_genres["genre"] == g, "movieID"].unique()
    df_movies["Genre {}".format(g)] = df_movies["id"].isin(mids).astype("int16")
df_movies.head()
# Podria ponderar por la cantidad de generos que tiene, si tiene 4 generos cada uno seria 0.25

,id,title,year,imdbID,rtID,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,1,Toy story,1995,114709,toy_story,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,113497,1068044-jumanji,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpy Old Men,1993,107050,grumpy_old_men,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,114885,waiting_to_exhale,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,113041,father_of_the_bride_part_ii,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Calculo para todos los usuarios

In [4]:
df_movies.head()

,id,title,year,imdbID,rtID,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,1,Toy story,1995,114709,toy_story,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,113497,1068044-jumanji,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpy Old Men,1993,107050,grumpy_old_men,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,114885,waiting_to_exhale,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,113041,father_of_the_bride_part_ii,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def perfil_por_usuario(USUARIO):

    # Voy a crea el perfil de un usuario
    df_ratings_generic = df_ratings[df_ratings["userID"] == USUARIO]
    df_ratings_generic.sort_values("rating").head()
    # Ahora lo que busco es condensar sus gustos de pelicula por los generos.
    # Mergeo al rating de un usuario el perfil de cada pelicula
    df_ratings_generic = df_ratings[df_ratings["userID"] == USUARIO].merge(df_movies, left_on="movieID", right_on="id")
    df_ratings_generic.sort_values("rating").head()

    # Perfil del USER 29202, la idea es generar el perfil para los usuarios.
    # Creo PERFIL como diccionario
    PERFIL ={}
    for c in df_ratings_generic.columns:
        # Me quedo solo con las columnas que empiezan con GENRE 
        if  c.startswith("userID"):
           PERFIL[c] = [USUARIO] 
        #if not c.startswith(("Genre", "userID")):
        if not c.startswith("Genre"):
            continue
        PERFIL[c] = [df_ratings_generic.loc[df_ratings_generic[c]==1, "rating"].mean() / 5.0]


    #print(PERFIL)
    return PERFIL

In [6]:
#IDs = ['29202','75','62354']
#df_addoc = df_ratings.loc[df_ratings["userID"].isin(IDs)]
df_addoc = df_ratings
#df_perfil = pd.DataFrame.from_dict(PERFIL).fillna(0)
#df_perfil.insert(0,'userID',USUARIO)
df_full_profile = pd.DataFrame()
for g in tqdm_notebook(df_addoc["userID"].unique()):
    #PERFIl_FULL = perfil_por_usuario(g)
    df_full_profile = df_full_profile.append(pd.DataFrame.from_dict(perfil_por_usuario(g)).fillna(0), ignore_index=True)
    #print(df_full_profile.tail(1))


In [7]:
df_full_profile.head()

,userID,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,Genre Romance,Genre Drama,Genre Action,Genre Crime,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,51198,0.798000,0.788889,0.766667,0.779000,0.796667,0.810811,0.833028,0.783824,0.865116,...,0.728571,0.804762,0.784211,0.9,0.883333,0.816667,0.840000,0.800000,0.733333,0.0
1,29202,0.503182,0.447059,0.433333,0.443613,0.492742,0.536408,0.566725,0.512500,0.534911,...,0.464516,0.548750,0.509357,0.6,0.653333,0.575556,0.504545,0.442857,0.556522,0.0
2,32283,0.694545,0.806452,0.758333,0.694751,0.694805,0.723832,0.739817,0.687004,0.728571,...,0.662162,0.742857,0.656566,0.8,0.828571,0.721212,0.700000,0.800000,0.769565,0.0
3,36495,0.677551,0.763636,0.742857,0.764198,0.631579,0.740541,0.755914,0.673034,0.767241,...,0.800000,0.725000,0.640000,0.0,0.500000,0.762500,0.800000,1.000000,0.600000,0.0
4,28350,0.666667,0.705769,0.658915,0.632463,0.656180,0.670940,0.669822,0.653191,0.635714,...,0.429412,0.615000,0.597561,0.8,0.000000,0.684211,0.641667,0.700000,0.635294,0.0


## En df_full_profile tengo el perfil de todos los usuarios y cada linea representa segun lo que califico un usuario que generos estan mas cerca o mas lejos de sus preferencia.
## Ahora lo que busco es cuales son las peliculas que estan mas cerca de casa usuario usando la Similitud Coseno (1 es super similar)

In [8]:
df_movies_adDoc = df_movies

In [42]:
# Dado un perfil de usuario, veo que pelicula es la mas afin a sus votaciones.

def pelicula_afin_a_perfil(userID, B, list_profile):
    columnas_genero = [c for c in df_movies_adDoc.columns if c.startswith("Genre")]
    
    similarityC_base=0
    movieID = 0
    #for index, row in tqdm_notebook(df_movies_adDoc.iterrows()):
    for index, row in (df_movies_adDoc.iterrows()):    
        movie = df_movies_adDoc.query("id == {}".format(row["id"]))
        A = movie[columnas_genero].values[0]
        similarityC = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
        #print(userID, row["id"], similarityC)
        if (similarityC_base < similarityC):
            similarityC_base = similarityC
            movieID = row["id"]
        #RANKING[index] = [row["id"], similarityC_base]
           
    user_row = dict()
    user_row["userID"] = userID
    user_row["movieID"] = movieID
    user_row["similarityC"] = similarityC_base
    list_profile.append(user_row)
    return list_profile

In [50]:
df_full_profileCut = df_full_profile[0:1]

In [44]:
# Calculo la similitud coseno para cada perfil contra todas las peliculas
profile_movieSelection = []
COUNT=1
for g in tqdm_notebook(df_full_profileCut["userID"].unique()):
    B=df_full_profileCut.iloc[COUNT-1:COUNT, 1:].values[0]
    pelicula_afin_a_perfil(g,B,profile_movieSelection)
    COUNT=COUNT+1


In [51]:
#df_profile_movieSelection
df_profile_movieSelection = pd.DataFrame(profile_movieSelection)
df_profile_movieSelection.head()

,userID,movieID,similarityC
0,51198,51709,0.634122


### Con esto obtuve una pelicula para recomendarle a un usuario, basado en las peliculas que ya puntuo.

### a) Poner el promedio de rating de cada pelicula.


In [13]:
def write_submit(df_test,prediccion,fileName):
    submit = pd.DataFrame({'ID':df_test.ID})
    submit['rating'] = pd.DataFrame(np.around(prediccion, 2))
    submit.head()
    len(submit)
    submit.to_csv("./results/"+fileName+".csv", index=False)

In [14]:
df_ratings_addoc = df_ratings
df_ratings_addoc.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,1,51198,608,5.0,17,9,1997,23,40,20
1,2,51198,141,4.0,17,9,1997,23,41,58
2,3,51198,1073,4.0,17,9,1997,23,41,6
3,4,51198,356,4.0,17,9,1997,23,43,44
4,5,51198,1198,5.0,17,9,1997,23,44,35


In [15]:
movies_avg = pd.DataFrame()
movies_avg = df_ratings_addoc.groupby("movieID")["rating"].mean().reset_index()
movies_avg.columns=["movieID", "movie_avg"]
movies_avg

,movieID,movie_avg
0,1,3.732032
1,2,2.977077
2,3,2.854348
3,4,2.487179
4,5,2.774038
...,...,...
10041,65088,3.500000
10042,65091,4.000000
10043,65126,3.500000
10044,65130,2.500000


In [16]:
mediaGeneral = movies_avg["movie_avg"].mean()
df_ratings_avg = df_ratings_addoc.merge(movies_avg, on="movieID", how="left")
df_ratings_avg = df_ratings_avg.fillna(mediaGeneral)
df_ratings_test_avg = df_ratings_test.merge(movies_avg, on="movieID", how="left")
df_ratings_test_avg= df_ratings_test_avg.fillna(mediaGeneral)

In [17]:
print("train: ",len(df_ratings_avg),len(df_ratings_addoc))
print("test: ",len(df_ratings_test_avg),len(df_ratings_test))

train:  770089 770089
test:  85509 85509


## Regresion Lineal inicial: Train, split de df y calculo de MSE

In [18]:
df_train = df_ratings_avg
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
full_x = df_train.drop(['rating', 'ID'], axis=1)
full_y = df_train[['rating']]
X_train, X_test, y_train, y_test = train_test_split(full_x, full_y, test_size=0.3, random_state=0)
print("train: ",len(X_train),len(y_train))
print("test:  ",len(X_test),len(y_test))
# Creo regresion lineal
model_avg = LinearRegression()
# Entreno el modelo usando el conjunto de TRAIN
model_avg.fit(X_train, y_train)
# Hago predicciones usando el TEST 
prediction_avg = model_avg.predict(X_test)
#ECM
print('Mean squared error: %.4f' % mean_squared_error(y_test, prediction_avg))


train:  539062 539062
test:   231027 231027
Mean squared error: 0.7585


## Regresion lineal: Test y escritura submit

In [19]:
df_x = df_ratings_test_avg.drop(['rating', 'ID'], axis=1)
modelFull_avg = LinearRegression()
# Entreno el modelo usando el conjunto de TRAIN
modelFull_avg.fit(X_train, y_train)

# Hago predicciones usando el TEST 
predFull_avg = modelFull_avg.predict(df_x)
print(len(df_ratings_test_avg.ID),len(predFull_avg))

85509 85509


In [20]:
write_submit(df_ratings_test_avg,predFull_avg, "ignacio_submit2")

In [21]:
df_ratings_test_avg.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,movie_avg
0,9,29202,1608,3.213463,18,9,1997,0,53,58,3.055254
1,18,32283,1485,3.213463,18,9,1997,15,45,18,3.070827
2,34,32283,30,3.213463,18,9,1997,16,36,38,3.777778
3,39,32283,1545,3.213463,18,9,1997,16,47,39,4.000000
4,50,51198,1258,3.213463,20,9,1997,0,11,21,3.964068


# *******************************************************************************************************
## LeaderBoard: 0.88102

### b) Poner el promedio de rating por mes para cada mes.

In [22]:
df_ratings_avg.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,movie_avg
0,1,51198,608,5.0,17,9,1997,23,40,20,4.020387
1,2,51198,141,4.0,17,9,1997,23,41,58,3.406467
2,3,51198,1073,4.0,17,9,1997,23,41,6,3.689828
3,4,51198,356,4.0,17,9,1997,23,43,44,3.931526
4,5,51198,1198,5.0,17,9,1997,23,44,35,4.136558


In [23]:
day_avg = pd.DataFrame()
day_avg = df_ratings_avg.groupby(["date_day","date_month", "date_year"])["rating"].mean().reset_index()
day_avg.columns=[ "date_day","date_month", "date_year", "day_avg"]
day_avg

,date_day,date_month,date_year,day_avg
0,1,1,1998,3.750000
1,1,1,1999,3.500000
2,1,1,2000,3.666667
3,1,1,2001,3.573333
4,1,1,2002,3.222973
...,...,...,...,...
3691,31,12,2004,3.658363
3692,31,12,2005,3.644410
3693,31,12,2006,3.040541
3694,31,12,2007,3.349217


In [24]:
mediaGeneral_day_avg = day_avg["day_avg"].mean()
df_ratings_avg_day = df_ratings_avg.merge(day_avg, left_on=["date_day", "date_month","date_year"], right_on = ["date_day", "date_month","date_year"], how="left")
df_ratings_avg_day = df_ratings_avg_day.fillna(mediaGeneral_day_avg)
df_ratings_test_avg_day = df_ratings_test_avg.merge(day_avg, left_on=["date_day", "date_month","date_year"], right_on = ["date_day","date_month","date_year"], how="left")
df_ratings_test_avg_day = df_ratings_test_avg_day.fillna(mediaGeneral_day_avg)

In [25]:
print("train: ",len(df_ratings_avg),len(df_ratings_avg_day))
print("test: ",len(df_ratings_test_avg),len(df_ratings_test_avg_day))

train:  770089 770089
test:  85509 85509


In [26]:
    df_train = df_ratings_avg_day
    full_x = df_train.drop(['rating', 'ID'], axis=1)
    full_y = df_train[['rating']]
    X_train, X_test, y_train, y_test = train_test_split(full_x, full_y, test_size=0.3, random_state=0)
    print("train: ",len(X_train),len(y_train))
    print("test:  ",len(X_test),len(y_test))
    # Creo regresion lineal
    model_avg2 = LinearRegression()
    # Entreno el modelo usando el conjunto de TRAIN
    model_avg2.fit(X_train, y_train)
    # Hago predicciones usando el TEST 
    prediction_avg2 = model_avg2.predict(X_test)
    #ECM
    print('Mean squared error: %.4f' % mean_squared_error(y_test, prediction_avg2))

train:  539062 539062
test:   231027 231027
Mean squared error: 0.7197


In [27]:
df_x = df_ratings_test_avg_day.drop(['rating', 'ID'], axis=1)
modelFull_avg2 = LinearRegression()
# Entreno el modelo usando el conjunto de TRAIN
modelFull_avg2.fit(X_train, y_train)

# Hago predicciones usando el TEST 
predFull_avg2 = modelFull_avg2.predict(df_x)
print(len(df_ratings_test_avg.ID),len(predFull_avg2))

85509 85509


In [28]:
write_submit(df_ratings_test_avg,predFull_avg2, "ignacio_submit3")

## LeaderBoard: 0.88102

## C) Agrego la similitud coseno entre las peliculas que puntuo y su perfil

In [29]:
#df_full_profile.to_csv("./results/df_full_profile.csv", index=False)
#df_ratings_avg_day.to_csv("./results/df_train_ab.csv", index=False)
#df_ratings_test_avg_day.to_csv("./results/df_test_ab.csv", index=False)

df_train_ab = pd.read_csv("./results/df_train_ab.csv")
df_test_ab = pd.read_csv("./results/df_test_ab.csv")
df_full_profile = pd.read_csv("./results/df_full_profile.csv")

In [30]:
df_full_profile.head()

,userID,Genre Adventure,Genre Animation,Genre Children,Genre Comedy,Genre Fantasy,Genre Romance,Genre Drama,Genre Action,Genre Crime,...,Genre Horror,Genre Mystery,Genre Sci-Fi,Genre IMAX,Genre Documentary,Genre War,Genre Musical,Genre Film-Noir,Genre Western,Genre Short
0,51198,0.798000,0.788889,0.766667,0.779000,0.796667,0.810811,0.833028,0.783824,0.865116,...,0.728571,0.804762,0.784211,0.9,0.883333,0.816667,0.840000,0.800000,0.733333,0.0
1,29202,0.503182,0.447059,0.433333,0.443613,0.492742,0.536408,0.566725,0.512500,0.534911,...,0.464516,0.548750,0.509357,0.6,0.653333,0.575556,0.504545,0.442857,0.556522,0.0
2,32283,0.694545,0.806452,0.758333,0.694751,0.694805,0.723832,0.739817,0.687004,0.728571,...,0.662162,0.742857,0.656566,0.8,0.828571,0.721212,0.700000,0.800000,0.769565,0.0
3,36495,0.677551,0.763636,0.742857,0.764198,0.631579,0.740541,0.755914,0.673034,0.767241,...,0.800000,0.725000,0.640000,0.0,0.500000,0.762500,0.800000,1.000000,0.600000,0.0
4,28350,0.666667,0.705769,0.658915,0.632463,0.656180,0.670940,0.669822,0.653191,0.635714,...,0.429412,0.615000,0.597561,0.8,0.000000,0.684211,0.641667,0.700000,0.635294,0.0


In [31]:
List_train = df_train_ab["userID"].unique()
List_test = df_test_ab["userID"].unique()

In [32]:
check =  all(item in List_train for item in List_test) 
if check is True:
    print("La lista {} contiene todos los elementos de la lista {}".format(List_train, List_test))    
else :
    print("No, List_train no contiene todos los elementos de List_test.")

La lista [51198 29202 32283 ... 62354 62394 62847] contiene todos los elementos de la lista [29202 32283 51198 ... 62354 62394 62847]


In [33]:
userTrain = df_train_ab.groupby("userID")
userTrain

In [34]:
len(df_train_ab[df_train_ab["userID"] == 51198].movieID)
len(df_train_ab["userID"].unique())

2113

In [61]:
# Dado un perfil de usuario, veo que pelicula es la mas afin a sus votaciones.

def subsetMovies_afin_a_perfil(userID, B, list_profile, df_movieList):
    columnas_genero = [c for c in df_movieList.columns if c.startswith("Genre")]
    
    similarityC_base=0
    movieID = 0
    #for index, row in tqdm_notebook(df_movieList.iterrows()):
    for index, row in (df_movieList.iterrows()):    
        movie = df_movieList.query("id == {}".format(row["id"]))
        A = movie[columnas_genero].values[0]
        similarityC = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))
        #print(userID, row["id"], similarityC)
        if (similarityC_base < similarityC):
            similarityC_base = similarityC
            movieID = row["id"]
        #RANKING[index] = [row["id"], similarityC_base]
           
    user_row = dict()
    user_row["userID"] = userID
    #user_row["movieID"] = movieID
    user_row["similarityC"] = similarityC_base
    list_profile.append(user_row)
    return list_profile

In [69]:
df_movies_adDoc = df_movies

In [73]:
similarityC_userID = []
# 2113 userID diferentes
# Calculo la similitud coseno para cada perfil contra todas las peliculas
for userID, group in tqdm_notebook(df_train_ab.groupby("userID")):
    movieList = []
    # Creo una lista con todos las peliculas para un usuario
    for row_index, row in group.iterrows():
        movieList.append(row.movieID)
    df_movies_subset = df_movies_adDoc[df_movies_adDoc["id"].isin(movieList)]
    B=df_full_profile.loc[df_full_profile["userID"] == 32283 ].drop(["userID"], axis=1).values[0]
    subsetMovies_afin_a_perfil(userID, B, similarityC_userID, df_movies_subset)

In [99]:
#similarityC_userID
#df_similarityC_userID = pd.DataFrame(similarityC_userID)
#len(df_similarityC_userID)
df_similarityC_userID = pd.read_csv("./results/df_similarityC_userID.csv")

df_train_ab = pd.read_csv("./results/df_train_ab.csv")
df_test_ab = pd.read_csv("./results/df_test_ab.csv")


In [100]:
#df_similarityC_userID["similarityC"] = 5 * df_similarityC_userID["similarityC"]
df_similarityC_userID

,userID,similarityC
0,75,2.483256
1,78,2.724130
2,127,2.415406
3,170,2.790629
4,175,3.025338
...,...,...
2108,71497,2.822414
2109,71509,3.025338
2110,71525,2.790629
2111,71529,2.790629


In [89]:
df_train_ab.head()

,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,movie_avg,day_avg
0,1,51198,608,5.0,17,9,1997,23,40,20,4.020387,4.142857
1,2,51198,141,4.0,17,9,1997,23,41,58,3.406467,4.142857
2,3,51198,1073,4.0,17,9,1997,23,41,6,3.689828,4.142857
3,4,51198,356,4.0,17,9,1997,23,43,44,3.931526,4.142857
4,5,51198,1198,5.0,17,9,1997,23,44,35,4.136558,4.142857


In [83]:
df_train_abc = df_train_ab.merge(df_similarityC_userID, on="userID", how="left")
df_test_abc = df_test_ab.merge(df_similarityC_userID, on="userID", how="left")
print(len(df_train_ab), len(df_train_abc))
print(len(df_test_ab), len(df_test_abc))

770089 770089
85509 85509


,ID,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second,movie_avg,day_avg,similarityC
0,9,29202,1608,3.213463,18,9,1997,0,53,58,3.055254,3.379310,0.605068
1,18,32283,1485,3.213463,18,9,1997,15,45,18,3.070827,3.379310,0.565422
2,34,32283,30,3.213463,18,9,1997,16,36,38,3.777778,3.379310,0.565422
3,39,32283,1545,3.213463,18,9,1997,16,47,39,4.000000,3.379310,0.565422
4,50,51198,1258,3.213463,20,9,1997,0,11,21,3.964068,3.941176,0.565422
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85504,855525,24708,64249,3.213463,4,1,2009,23,29,16,3.111111,3.198835,0.605068
85505,855545,61240,1253,3.213463,5,1,2009,0,58,34,3.934066,3.704545,0.558789
85506,855551,36559,52604,3.213463,5,1,2009,1,2,3,3.655405,3.704545,0.558789
85507,855566,18206,837,3.213463,5,1,2009,1,18,40,3.246875,3.704545,0.611448
